In [47]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

In [43]:
# Function to get company information from a single page
def scrape_page(page_url):
    response = requests.get(page_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    company_list = []
    # Extracting information from the page
    if len(soup.find_all('div', class_='MuiGrid-root MuiGrid-item MuiGrid-grid-xs-12 MuiGrid-grid-sm-6 css-rpybyc'))>0:
        for company_card in soup.find_all('div', class_='MuiGrid-root MuiGrid-item MuiGrid-grid-xs-12 MuiGrid-grid-sm-6 css-rpybyc'):
            email = company_card.find('p', class_='MuiTypography-root MuiTypography-body1 css-1l3ldki').text
            if len(company_card.find_all('p', class_='MuiTypography-root MuiTypography-body1 css-1vlcel7'))==2:
                name, category = [ele.text for ele in company_card.find_all('p', class_='MuiTypography-root MuiTypography-body1 css-1vlcel7')]
            else:
                name = company_card.find('p', class_='MuiTypography-root MuiTypography-body1 css-1vlcel7')
                category = ''
            company_info = {
                'name': name,
                'email': email,
                'category': category
            }
            company_list.append(company_info)
        return company_list
    else:
        return 0/0

# Function to scrape all pages for a given letter
def scrape_letter(letter):
    base_url = 'https://skrapp.io/directory'
    letter_url = f'{base_url}/{letter}'

    response = requests.get(f'{letter_url}/1')
    soup = BeautifulSoup(response.text, 'html.parser')

    # Get the number of pages for the letter
    page_num = 1
    endOfPages = False
    all_companies = []
    while (not endOfPages):
        try:
            page_url = f'{letter_url}/{page_num}'
            companies_on_page = scrape_page(page_url)
            all_companies.extend(companies_on_page)
            # Sleep for a short while to avoid overloading the server
            time.sleep(.5)
            page_num+=1
        except:
            endOfPages = True
    return all_companies

# Main function to scrape all letters
def scrape_all():
    all_data = []

    # Iterate through each letter
    for letter in 'abcdefghijklmnopqrstuvwxyz':
        print(letter)
        letter_data = scrape_letter(letter)
        all_data.extend(letter_data)

    return all_data

In [44]:
company_emails=scrape_all()

a
b
c
d
e
f
g
h
i
j
k
l
m
n
o
p
q
r
s
t
u
v
w
x
y
z


In [96]:
df_company_emails = pd.DataFrame(company_emails)
for col in ['name', 'email', 'category']:
    df_company_emails[col] = df_company_emails[col].apply(lambda x: x.text if type(x) != str else x)

In [97]:
"""
gmail.com - not Hamlin
yahoo.com - remove
yandex.com - remove
zoho.com - remove
telenet.be - remove
earthlink.net - remove
shaw.ca - remove
rcn.com - remove
rogers.com - remove
nus.edu.sg - remove, is educational institution
uol.com.br - remove
"""

'\ngmail.com - not Hamlin\nyahoo.com - remove\nyandex.com - remove\nzoho.com - remove\ntelenet.be - remove\nearthlink.net - remove\nshaw.ca - remove\nrcn.com - remove\nrogers.com - remove\nnus.edu.sg - remove, is educational institution\nuol.com.br - remove\n'

In [107]:
email_list = ['gmail.com', 'yahoo.com','yandex.com','zoho.com' ,'telenet.be','earthlink.net','shaw.ca',
              'rcn.com','rogers.com','uol.com.br']
df_company_emails = df_company_emails[~df_company_emails['email'].isin(email_list)]

In [109]:
df_company_emails.to_csv('data/inputs/more_company_domains.csv')